In [1]:
%load_ext autoreload


In [2]:
import numpy as np
import pandas as pd
from pycaret.time_series import TSForecastingExperiment

In [11]:
%autoreload
import sys
sys.path.append(sys.path[0][0:-4] + 'mikael')
from pipeline import Pipin
pipin = Pipin()

In [4]:
global_fig_settings = {
    # "renderer": "notebook",
    "renderer": "png",
    "width": 1000,
    "height": 600/13*52,
}

In [5]:
data0 = pipin.get_combined_datasets(timeSeries=True, randomize=False)

In [12]:
get_data_a = pipin.get_data("A", timeSeries=True)
get_data_b = pipin.get_data("B", timeSeries=True)
get_data_c = pipin.get_data("C", timeSeries=True)

In [13]:
# groupe by index
# merge the two data sets together
# merged_df = pd.merge(categorical_df, numeric_df, left_index=True, right_index=True)
merged_df = data0.copy()
df_a = get_data_a.copy()
df_b = get_data_b.copy()
df_c = get_data_c.copy()

In [20]:
# We want to forecast the next 12 months of data and we will use 3 fold cross-validation to test the models.
fh = 24  # or alternately fh = np.arange(1,13)
target = "pv_measurement"
eda = TSForecastingExperiment()
ignore_features = pipin.get_irrelevant_features() #+ pipin.get_categorical_features(merged_df)
setup_a = eda.setup(
    data=df_a,
    target=target,
    fh=fh,
    ignore_features=ignore_features,
    # numeric_imputation_target=None,
    numeric_imputation_target="ffill",
    numeric_imputation_exogenous="ffill",
    fig_kwargs=global_fig_settings,
    session_id=42,
    # normalize
)

,Description,Value
0,session_id,42
1,Target,pv_measurement
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(34274, 46)"
5,Transformed data shape,"(34274, 46)"
6,Transformed train set shape,"(34250, 46)"
7,Transformed test set shape,"(24, 46)"
8,Rows with missing values,13.3%
9,Fold Generator,ExpandingWindowSplitter


In [15]:
best_model = eda.compare_models(n_select=3)


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
xgboost_cds_dt,Extreme Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.7756,0.6323,305.5440,523.3456,8182060578003957.0000,0.8730,0.6786,1.1200
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,0.8069,0.6541,317.8927,541.3867,971060060350354.0000,0.8425,0.5972,13.7933
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,0.8378,0.6771,330.0541,560.4156,2166842410335710.0000,0.8332,0.5369,3.9033
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.8718,0.6790,343.4606,561.9859,15192938317341530.0000,0.8942,0.6252,12.9400
auto_arima,Auto ARIMA,0.8944,0.6778,352.3768,561.0195,52299829241947720.0000,0.9179,0.6166,4.9833
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,0.8988,0.6650,354.1254,550.4055,84681363679339632.0000,0.9847,0.6092,0.4900
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,0.9018,0.6691,355.3180,553.7877,81326985756314112.0000,0.9780,0.6031,1.4433
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,0.9089,0.6759,358.0850,559.4361,77981209413856016.0000,0.9619,0.5929,0.4300
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,0.9103,0.6767,358.6465,560.0898,77628675553942144.0000,0.9625,0.5914,1.0533
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,0.9145,0.6742,360.3096,558.0122,88550593592763760.0000,0.9684,0.5850,0.4800


In [21]:
# print(best_model)
# eda.models()
# BaseCdsDtForecaster RandomForestRegressor rf_cds_dt
# BaseCdsDtForecaster ExtraTreesRegressor et_cds_dt
# BaseCdsDtForecaster GradientBoostingRegressor lightgbm_cds_dt

model_xgb = setup_a.create_model("xgboost_cds_dt")


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2023-04-26 23:00,0.3344,0.2744,131.7756,227.1458,4175942954205094.5000,0.8602,0.7465
1,2023-04-27 23:00,2.1370,1.7296,841.7722,1431.3284,5097918284020016.0000,0.9349,0.0388
2,2023-04-28 23:00,0.2467,0.2051,97.2552,169.8594,5734933987176321.0000,0.8196,0.9277
Mean,NaT,0.9060,0.7364,356.9344,609.4445,5002931741800477.0000,0.8716,0.5710
SD,NaT,0.8711,0.7029,343.1217,581.6301,639989631490641.7500,0.0478,0.3835


In [22]:
model_xgb_tuned = setup_a.tune_model(model_xgb)


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2023-04-26 23:00,0.3902,0.3095,153.7547,256.2168,16609167458478710.0000,0.9243,0.6775
1,2023-04-27 23:00,1.8545,1.4997,730.5222,1241.0827,17957371256391110.0000,0.9492,0.2774
2,2023-04-28 23:00,0.2659,0.2409,104.8235,199.4948,21058358256389268.0000,0.7797,0.9002
Mean,NaT,0.8369,0.6834,329.7001,565.5981,18541632323753028.0000,0.8844,0.6184
SD,NaT,0.7214,0.5779,284.1271,478.2007,1862765942566620.2500,0.0748,0.2577


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   21.3s finished


In [24]:
final_model = setup_a.finalize_model(model_xgb_tuned)

In [27]:
test_data = pipin.get_data("A", timeSeries=True)

In [34]:
# test = df_a.drop(columns=["pv_measurement"])
# y_predict = setup_a.predict_model(final_model, X=test)

In [35]:
eda.plot_model(estimator=final_model, fig_kwargs={"height":1200, "width":1000})

ValueError: Model was trained with exogenous variables but you have not passed any for predictions. Please pass exogenous variables to make predictions.

In [14]:
# tuned_model = eda.tune_model(model)
# eda.plot_model(estimator=tuned_model)

In [15]:
# final_model = eda.finalize_model(model)

# eda.predict_model
# eda.plot_model(final_model, )
# eda.predict_model(final_model)

In [50]:
merged_df


,dew_or_rime:idx,is_day:idx,is_in_shadow:idx,precip_type_5min:idx,snow_drift:idx,location:idx,super_cooled_liquid_water:kgm2,fresh_snow_12h:cm,precip_5min:mm,fresh_snow_6h:cm,...,month,elevation:m,clear_sky_energy_1h:J,diffuse_rad_1h:J,snow_depth:cm,prob_rime:p,dew_point_2m:K,pressure_50m:hPa,snow_density:kgm3,snow_melt_10min:mm
index,,,,,,,,,,,,,,,,,,,,,
2019-06-02 22:00:00,0.0,0.0,1.0,0.0,0.0,A,0.000,0.0,0.0,0.0,...,6.0,6.0,0.000000,0.000000,0.0,0.0,280.299988,999.775024,NaN,0.0
2019-06-02 23:00:00,0.0,0.0,1.0,0.0,0.0,A,0.000,0.0,0.0,0.0,...,6.0,6.0,0.000000,0.000000,0.0,0.0,280.299988,998.650024,NaN,0.0
2019-06-03 00:00:00,0.0,0.0,1.0,0.0,0.0,A,0.000,0.0,0.0,0.0,...,6.0,6.0,0.000000,0.000000,0.0,0.0,280.649994,997.974976,NaN,0.0
2019-06-03 01:00:00,0.0,0.0,1.0,0.0,0.0,A,0.000,0.0,0.0,0.0,...,6.0,6.0,208.649994,526.775024,0.0,0.0,281.674988,997.449951,NaN,0.0
2019-06-03 02:00:00,0.0,1.0,0.0,0.0,0.0,A,0.000,0.0,0.0,0.0,...,6.0,6.0,32468.150391,22068.949219,0.0,0.0,282.500000,996.500000,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2033-04-30 19:00:00,0.0,0.0,1.0,0.0,0.0,C,0.000,0.0,0.0,0.0,...,4.0,24.0,84010.148438,54774.000000,0.0,0.0,272.024994,1005.275024,250.0,0.0
2033-04-30 20:00:00,0.0,0.0,1.0,0.0,0.0,C,0.000,0.0,0.0,0.0,...,4.0,24.0,2206.800049,4984.049805,0.0,0.0,271.950012,1005.250000,250.0,0.0
2033-04-30 21:00:00,0.0,0.0,1.0,0.0,0.0,C,0.025,0.0,0.0,0.0,...,4.0,24.0,0.000000,0.000000,0.0,0.0,271.899994,1005.125000,250.0,0.0


In [71]:
from pathlib import Path
# test_data = pipin.get_combined_test_data(True)
test_data = pipin.get_combined_test_data(time_series=True)
# test_data = pipin.get_test_data("A")
test_data = test_data[list(set(merged_df.columns) - {"pv_measurement"})]
# get empty rows from test data
# empty_rows = test_data[test_data.isnull().any(axis=1)]
test_data

,super_cooled_liquid_water:kgm2,location:idx,fresh_snow_12h:cm,precip_5min:mm,snow_drift:idx,fresh_snow_6h:cm,snow_water:kgm2,air_density_2m:kgm3,t_1000hPa:K,fresh_snow_3h:cm,...,clear_sky_energy_1h:J,diffuse_rad_1h:J,snow_depth:cm,prob_rime:p,dew_point_2m:K,pressure_50m:hPa,snow_density:kgm3,snow_melt_10min:mm,is_in_shadow:idx,is_day:idx
index,,,,,,,,,,,,,,,,,,,,,
2023-05-01 00:00:00,0.000,A,0.0,0.0,0.0,0.0,0.0,1.28675,273.799988,0.0,...,0.000000,0.000000,0.0,0.0,271.650024,1006.799988,NaN,0.0,1.0,0.0
2023-05-01 01:00:00,0.000,A,0.0,0.0,0.0,0.0,0.0,1.28600,273.799988,0.0,...,0.000000,0.000000,0.0,0.0,271.450012,1006.299988,NaN,0.0,1.0,0.0
2023-05-01 02:00:00,0.000,A,0.0,0.0,0.0,0.0,0.0,1.28375,273.849976,0.0,...,0.000000,0.000000,0.0,0.0,271.049988,1005.799988,NaN,0.0,1.0,0.0
2023-05-01 03:00:00,0.000,A,0.0,0.0,0.0,0.0,0.0,1.28200,273.899994,0.0,...,10124.424805,16845.226562,0.0,0.0,270.649994,1005.224976,NaN,0.0,0.0,1.0
2023-05-01 04:00:00,0.000,A,0.0,0.0,0.0,0.0,0.0,1.28100,273.924988,0.0,...,141748.593750,102209.703125,0.0,0.0,270.375000,1004.750000,NaN,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-03 19:00:00,0.100,C,0.0,0.0,0.0,0.0,0.0,1.19675,286.850006,0.0,...,475981.375000,162567.531250,0.0,0.0,281.700012,983.400024,NaN,0.0,0.0,1.0
2023-07-03 20:00:00,0.000,C,0.0,0.0,0.0,0.0,0.0,1.20000,286.450012,0.0,...,183261.343750,82875.351562,0.0,0.0,282.024994,983.849976,NaN,0.0,0.0,1.0
2023-07-03 21:00:00,0.000,C,0.0,0.0,0.0,0.0,0.0,1.20350,285.950012,0.0,...,36831.074219,27210.250000,0.0,0.0,282.350006,984.150024,NaN,0.0,1.0,0.0


In [48]:
def get_unique_filename(base_name, folder = "submissions"):
    end = ""
    if folder == "models":
        end = ".pkl"
    
    file_path = Path(folder) / f"{base_name}.csv{end}"
    count = 1
    while file_path.exists():
        file_path = Path(folder) / f"{base_name}_{count}.csv{end}"
        count += 1
    return str(file_path)

In [72]:
# res_goose = predict_model(lightgbm, test_data)
res_stack = eda.predict_model(final_model, X=test_data)
res_stack = res_stack[["prediction_label"]].rename(columns={"prediction_label": "prediction"}).rename_axis(index="id")
# res_stack.to_csv(get_unique_filename('stack_gbm_et_rf'))
# save_model(stacker, model_name=get_unique_filename('stack_gbm_et_rf', "models"))

TypeError: X must be either None, or in an sktime compatible format, of scitype Series, Panel or Hierarchical, for instance a pandas.DataFrame with sktime compatible time indices, or with MultiIndex and last(-1) level an sktime compatible time index. See the forecasting tutorial examples/01_forecasting.ipynb, or the data format tutorial examples/AA_datatypes_and_datasets.ipynbIf you think X is already in an sktime supported input format, run sktime.datatypes.check_raise(X, mtype) to diagnose the error, where mtype is the string of the type specification you want for X. Possible mtype specification strings are as follows. "For Series scitype: ['pd.Series', 'pd.DataFrame', 'np.ndarray']. , "For Panel scitype: ['pd-long', 'nested_univ', 'numpy3D', 'pd-multiindex', 'df-list']. , "For Hierarchical scitype: ['pd_multiindex_hier']. 